# Challenge Empresarial:
## Crear un script que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aldomunaretto/immune_generative_ai/blob/main/notebooks/02_crear_brochure.ipynb)

### Importamos las librerias necesarias

In [2]:
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

### Cargamos las variables del fichero .env

In [ ]:
load_dotenv()

### Creamos el objeto para la conexión con la API de OpenAI.

In [ ]:
openai = OpenAI()

### Creamos una nueva clase Website para scrappear sitios web incluyendo los enlaces

In [ ]:
class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [ ]:
aegon = Website("https://www.aegon.es")
print(aegon.get_contents())
aegon.links

## Primer paso: hacer que GPT-5-nano determine qué enlaces son relevantes

### Usaremos una primera llamada a GPT-5-nano para obtener los enlaces en una página web y responder en JSON estructurado.

Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

<u>Nota al margen:</u> existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación pero dejaremso esto para más adelante.

In [ ]:
link_system_prompt = """Se te proporciona una lista de enlaces que se encuentran en una página web.
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa,
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de servicios.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Seguros", "url": "https://otra.url.completa/seguros"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [ ]:
def get_links_user_prompt(website):
    links_list = '\n'.join(website.links)
    user_prompt = f"""Aquí hay una lista de enlaces de la página web {website.url} - 
    Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON.
    No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
    Links (puede que algunos sean links relativos): {links_list}"""
    return user_prompt

In [ ]:
print(get_links_user_prompt(aegon))

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.responses.create(
        model='gpt-5-nano',
        input=[
            {"role": "developer", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        text={"format": {"type": "json_object"}}
    ) 
    return json.loads(response.output_text)

In [ ]:
get_links("https://www.aegon.es")

In [ ]:
LINKS_SCHEMA = {
    "name": "links_output",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "links": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "type": {"type": "string"},
                        "url": {"type": "string"}
                    },
                    "required": ["type", "url"],
                    "additionalProperties": False
                }
            }
        },
        "required": ["links"],
        "additionalProperties": False
    }
}

def get_links_schema(url):
    website = Website(url)
    response = openai.responses.create(
        model='gpt-5-nano',
        input=[
            {"role": "developer", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        text={"format": {"type": "json_schema", **LINKS_SCHEMA}},
    )
    return json.loads(response.output_text)

In [ ]:
get_links_schema("https://www.aegon.es")

In [3]:
from pydantic import BaseModel, Field
from typing import List

class Link(BaseModel):
    type: str = Field(description="categoria del enlace de la página web, segun reelevancia para el folleto.")
    url: str = Field(
        description="URL completa del enlace (https://...).",
        pattern=r"^https?://\S+$",
    )
    
class LinksList(BaseModel):
    links: List[Link] = Field(description="Lista de enlaces.")

def get_links_structured(url: str) -> LinksList:
    website = Website(url)

    response = openai.responses.parse(
        model="gpt-5-nano",
        input=[
            {"role": "developer", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)},
        ],
        text_format=LinksList,
    )
    # return response.output_parsed ### devuelve el objeto ya validado (no necesitas json.loads) pero no nos permite visualizar el resultado como esperamos.
    return json.loads(response.output_text)

In [ ]:
get_links_structured("https://www.aegon.es")

In [ ]:
anthropic = Website("https://anthropic.com")
anthropic.links

In [ ]:
get_links("https://anthropic.com")

## Segundo paso: ¡creamos el folleto!

Reúne todos los detalles en otro mensaje para GPT-5-nano

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links_structured(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://anthropic.com"))

In [ ]:
system_prompt = """Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."""


### O puedes sustituir el prompt por el que se muestra a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

~~~python
system_prompt = """Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa
                   y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.
                   Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."""
~~~

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""Estás mirando una empresa llamada: {company_name}. Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes. 
                      Usa esta información para crear un breve folleto de la empresa en Markdown: {get_all_details(url)}"""
    user_prompt = user_prompt[:20_000]
    return user_prompt

In [ ]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

In [ ]:
def create_brochure(company_name, url):
    response = openai.responses.create(
        model='gpt-5-nano',
        input=[
            {"role": "developer", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    display(Markdown(response.output_text))

In [ ]:
create_brochure("Aegon España", "https://www.aegon.es/")

In [ ]:
create_brochure("Anthropic", "https://anthropic.com")

## En Streaming

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [ ]:
def stream_brochure(company_name, url):
    response_text = ""
    display_handle = display(Markdown(""), display_id=True)

    with openai.responses.stream(
        model="gpt-5-nano",
        input=[
            {"role": "developer", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
    ) as stream:
        for event in stream:
            if event.type == "response.output_text.delta":
                response_text += event.delta
                cleaned = response_text.replace("```", "").replace("markdown", "")
                update_display(Markdown(cleaned), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Anthropic", "https://anthropic.com")

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
stream_brochure("Aegon España", "https://www.aegon.es/")

## LM Studio

In [ ]:
import lmstudio as lms

def get_url_lmstudio(url):
    website = Website(url)
    model = lms.llm("openai/gpt-oss-20b")
    result = model.respond(link_system_prompt + get_links_user_prompt(website))

    return result

print(get_url_lmstudio("https://www.aegon.es"))


## Gemini

In [ ]:
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from google import genai
from pydantic import BaseModel, Field, HttpUrl
from typing import List

load_dotenv()

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"
    
link_system_prompt = """Se te proporciona una lista de enlaces que se encuentran en una página web.
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa,
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de servicios.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Seguros", "url": "https://otra.url.completa/seguros"}
    ]
}
"""

def get_links_user_prompt(website):
    links_list = '\n'.join(website.links)
    user_prompt = f"""Aquí hay una lista de enlaces de la página web {website.url} - 
    Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON.
    No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
    Links (puede que algunos sean links relativos): {links_list}"""
    return user_prompt


client = genai.Client()

class Link(BaseModel):
    type: str = Field(description="categoria del enlace de la página web, segun reelevancia para el folleto.")
    url: HttpUrl = Field(description="URL completa del enlace.")
    
class LinksList(BaseModel):
    links: List[Link] = Field(description="Lista de enlaces.")

def get_links_google(url):
    website = Website(url)
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=[link_system_prompt, get_links_user_prompt(website)],
        config={
            "response_mime_type": "application/json",
            "response_json_schema": LinksList.model_json_schema(),
        },
    )

    return LinksList.model_validate_json(response.text).model_dump()

def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links_google(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

brochure_system_prompt = """Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."""

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""Estás mirando una empresa llamada: {company_name}. Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes. 
                      Usa esta información para crear un breve folleto de la empresa en Markdown: {get_all_details(url)}"""
    return user_prompt
    
def create_brochure_google(company_name, url, stream : bool = False):
    if stream:
        response = client.models.generate_content_stream(
            model='gemini-3-flash-preview',
            contents=[brochure_system_prompt,get_brochure_user_prompt(company_name, url)]
        )
        handle = display(Markdown("Cargando folleto..."), display_id=True)
        full_text = ""
        
        for chunk in response:
            full_text += chunk.text
            handle.update(Markdown(full_text))
    else:
        response = client.models.generate_content(
            model='gemini-3-flash-preview',
            contents=[brochure_system_prompt,get_brochure_user_prompt(company_name, url)]
        )
        result = response.text
        display(Markdown(result))

In [ ]:
create_brochure_google("Aegon España", "https://www.aegon.es")


In [ ]:
create_brochure_google("Aegon España", "https://www.aegon.es", stream=True)